---

## Benchmark : Quel est le meilleur modèle pour la prédiction de la série temporelle globale ?

---

### Rappel des modèles :
- **SARIMAX**
- **Prophet** (Facebook) + version Online
- **LSTM**
- **XGBoost**
- **Random Forest**


In [35]:
import os 
import json
import pandas as pd
import plotly_express as px

file_path = "./results_selected.json"

In [36]:
# --Tables--
series_train = pd.read_csv(os.path.join("..", "models", "series_train.csv"))
series_test = pd.read_csv(os.path.join("..", "models", "series_test.csv"))
series_test.head()

,date,sales,onpromotion,oil_price,holiday,Test_Forecast_Sarimax,Test_Forecast_prophet,test_Forecast_LSTM,test_Forecast_XGB,test_Forecast_RF,test_Forecast_prophet_online
0,2016-01-01,1.643339e+04,159,37.076647,1,8.832907e+05,8.467677e+05,NaN,334607.97,386969.305608,690101.175402
1,2016-01-02,1.066677e+06,1832,36.970000,0,6.229928e+05,1.013931e+06,NaN,334607.97,825807.859319,869241.978477
2,2016-01-03,1.226736e+06,2936,36.863353,0,1.115351e+06,1.064571e+06,NaN,334607.97,866412.981765,918947.551836
3,2016-01-04,9.559569e+05,1741,36.810000,0,9.056684e+05,8.313662e+05,NaN,334607.97,838459.230084,670285.418443
4,2016-01-05,8.353204e+05,3950,35.970000,0,9.304216e+05,8.027423e+05,NaN,334607.97,935594.109989,637981.921649


In [37]:
series_test.columns

Index(['date', 'sales', 'onpromotion', 'oil_price', 'holiday',
       'Test_Forecast_Sarimax', 'Test_Forecast_prophet', 'test_Forecast_LSTM',
       'test_Forecast_XGB', 'test_Forecast_RF',
       'test_Forecast_prophet_online'],
      dtype='object')

In [38]:
Forecasts = ['sales',
       'Test_Forecast_Sarimax', 'Test_Forecast_prophet', 'test_Forecast_LSTM',
       'test_Forecast_XGB', 'test_Forecast_RF',
       'test_Forecast_prophet_online']

In [39]:
fig = px.line(series_test, x='date', y= Forecasts,title="Predictions du test par nos modeles")
fig.show()

Visuel interactif : cliquez sur les courbes pour une analyse approfondie

### 🔍 Trouvons les meilleurs modèles selon leur NRMSE

#### 🧮 Formule du NRMSE :
$$
NRMSE = \frac{\sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}}{y_{max} - y_{min}}
$$

#### Pourquoi le NRMSE est utile ? 🤔
- Le **NRMSE (Normalized Root Mean Square Error)** est une version normalisée de l'erreur quadratique moyenne. 
- En normalisant par l'amplitude des données $(y_{max} - y_{min})$, il permet une comparaison plus juste entre différents modèles, indépendamment de l'échelle des données.
- Cela nous donne une idée claire de la qualité de la prédiction relative à la variabilité des données.


In [40]:
with open(file_path, 'r') as file:
    data = json.load(file)

top_6_models = sorted(data, key=lambda x: x["NRMSE"])[:6] #trier

print("Top 5 modèles avec les meilleurs NRMSE :")
for i, model in enumerate(top_6_models, 1):
    print(f"{i}. {model['Model']} - NRMSE: {model['NRMSE']:.4f}")


Top 5 modèles avec les meilleurs NRMSE :
1. Prophet_online - NRMSE: 0.0643
2. LSTM - NRMSE: 0.0796
3. XGBoost - NRMSE: 0.0828
4. Prophet - NRMSE: 0.1094
5. Random_forest - NRMSE: 0.1111
6. SARIMAX - NRMSE: 0.1476


In [41]:
models = [model['Model'] for model in top_6_models]
nrmse_values = [model['NRMSE'] for model in top_6_models]

df_results = pd.DataFrame({
    "Model": models,
    "NRMSE": nrmse_values
})

fig = px.bar(
    df_results, 
    x="Model", 
    y="NRMSE", 
    title="Top 6 modèles par NRMSE",
    text="NRMSE", 
    labels={"Model": "Modèles", "NRMSE": "NRMSE"},
    color="NRMSE",
    color_continuous_scale="Blues"
)

fig.update_traces(texttemplate='%{text:.4f}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45, height=500, width=800)


fig.show()

> On affiche le NMRSE, ainsi la bar la plus petite est le meilleurs modele. 

### Sensibilité du MAPE et du $R^2$ aux petites valeurs

Le **MAPE** (Mean Absolute Percentage Error) et le **$R^2$** (Coefficient de détermination) sont très sensibles aux petites valeurs en raison de la division dans leurs formules.

#### Formule du MAPE
$$
\text{MAPE} = \frac{1}{n} \sum_{i=1}^n \left| \frac{y_i - \hat{y}_i}{y_i} \right| \times 100
$$

- Si $y_i$ est proche de zéro, la division par $y_i$ peut entraîner une valeur aberrante.

#### Formule du $R^2$
$$
R^2 = 1 - \frac{\text{SS}_{\text{residual}}}{\text{SS}_{\text{total}}}
$$

- Si les $y_i$ sont très proches de zéro, $\text{SS}_{\text{total}}$ devient fragile, et de petites erreurs dans les prédictions peuvent provoquer un $R^2$ négatif.

Nous avons des valeurs égales à 0 ou très faibles, ce qui fait exploser ces métriques. On regard alors le `NMRSE`